# Prediction of Vehicle Loan Default

## Artificial Intelligence for Finance Group Project

Moruf Bamigboye

Maxwell Djoumessi Kana

Hon Ching Yeung

Sébastien Palmas

In [1]:
import numpy as np 
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import confusion_matrix

In [2]:
# read the data

train_data = pd.read_csv("/Users/ching/Desktop/Class/Artificial Intelligence/Artificial Intelligence for Finance/project/train.csv")
train_data.set_index("UniqueID")

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
UniqueID,,,,,,,,,,,,,,,,,,,,,
420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626432,63213,105405,60.72,34,20700,48,1050,01-08-88,Salaried,26-10-18,...,0,0,4084,0,0,0,1yrs 9mon,3yrs 3mon,0,0
606141,73651,100600,74.95,34,23775,51,990,05-12-88,Self employed,23-10-18,...,0,0,1565,0,0,0,0yrs 6mon,0yrs 6mon,0,0
613658,33484,71212,48.45,77,22186,86,2299,01-06-76,Salaried,24-10-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0


In [3]:
# convert the date column to date time data type

train_data['Date.of.Birth'] = pd.to_datetime(train_data['Date.of.Birth'])
train_data['Date.of.Birth'] = train_data['Date.of.Birth'].map(dt.datetime.toordinal)

In [4]:
# check the correlation between variables

correlation = train_data.corr().sort_values(by = 'loan_default')
correlation

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,State_ID,...,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default
PERFORM_CNS.SCORE,-0.014016,0.011409,-0.047398,0.084993,-0.018049,-0.077029,0.073012,-0.161664,0.014670,-0.042323,...,0.035100,0.020128,0.024365,0.023953,0.071309,0.015655,0.346025,0.164745,0.152838,-0.057929
Aadhar_flag,-0.001372,-0.015589,-0.093716,0.106953,-0.036384,-0.081161,0.041251,-0.309285,-0.002003,-0.169882,...,0.007979,0.006420,0.007591,0.007540,0.014832,0.006273,0.059269,0.023823,0.027317,-0.041593
PRI.ACTIVE.ACCTS,-0.020862,0.039542,-0.021692,0.087929,-0.006403,-0.066930,0.065589,-0.117882,0.012185,-0.021061,...,0.037422,0.022018,0.026805,0.026325,0.192397,0.021210,0.702943,0.382584,0.195484,-0.041451
PRI.NO.OF.ACCTS,-0.013584,0.034461,-0.026798,0.086571,-0.010384,-0.070536,0.068371,-0.084687,0.021305,-0.008704,...,0.030364,0.020945,0.024877,0.024551,0.269546,0.023421,0.537622,0.314148,0.180744,-0.035456
NEW.ACCTS.IN.LAST.SIX.MONTHS,-0.003842,0.035880,-0.021968,0.083824,-0.008354,-0.053264,0.050079,-0.100080,-0.026619,-0.014942,...,0.057046,0.046465,0.055801,0.054886,0.127796,0.028160,1.000000,0.182769,0.264709,-0.029400
PRI.CURRENT.BALANCE,-0.007259,0.015713,-0.004356,0.026714,-0.004133,-0.022582,0.031722,-0.055948,0.019250,-0.006359,...,0.015225,0.019378,0.021179,0.021152,0.152499,0.006799,0.198566,0.244243,0.023861,-0.027386
manufacturer_id,-0.021514,0.095203,-0.005256,0.121003,-0.049957,-0.063916,1.000000,-0.104349,0.019486,0.026412,...,0.008152,0.003106,0.003206,0.003301,0.014116,0.003020,0.050079,0.019438,0.010012,-0.025039
PRI.SANCTIONED.AMOUNT,-0.002910,0.006294,-0.001180,0.009955,-0.003370,-0.012959,0.017242,-0.025843,0.016244,-0.004304,...,0.007967,0.010750,0.011859,0.011843,0.072591,0.003883,0.095074,0.122028,0.010743,-0.011304
PRI.DISBURSED.AMOUNT,-0.002563,0.006412,-0.001221,0.010177,-0.003412,-0.012950,0.017080,-0.025697,0.016221,-0.004129,...,0.007872,0.010682,0.011771,0.011757,0.073058,0.003806,0.094681,0.122650,0.010935,-0.011155
PRIMARY.INSTAL.AMT,-0.009321,0.002350,-0.004356,0.007358,0.004955,-0.011953,0.014116,0.002275,0.021960,0.015843,...,0.002066,0.010393,0.011448,0.011469,1.000000,0.010849,0.127796,0.090364,0.008967,-0.010616


In [5]:
# set up dependent and independent variables

train_data_y = train_data['loan_default']
train_data_y = train_data_y.astype('int64')
train_data_x = train_data.drop(['loan_default'], axis =1)

In [6]:
# establish train and test datasets

train_data_x, test_data, y_train, y_test = train_test_split(train_data_x, train_data_y, train_size = 0.8, test_size = 0.2)
train_data_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186523 entries, 184054 to 73018
Data columns (total 40 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             186523 non-null  int64  
 1   disbursed_amount                     186523 non-null  int64  
 2   asset_cost                           186523 non-null  int64  
 3   ltv                                  186523 non-null  float64
 4   branch_id                            186523 non-null  int64  
 5   supplier_id                          186523 non-null  int64  
 6   manufacturer_id                      186523 non-null  int64  
 7   Current_pincode_ID                   186523 non-null  int64  
 8   Date.of.Birth                        186523 non-null  int64  
 9   Employment.Type                      180328 non-null  object 
 10  DisbursalDate                        186523 non-null  object 
 11  State_ID 

In [7]:
# fill up any missing value

SI = SimpleImputer(strategy = "most_frequent", missing_values = np.nan)
x_train = pd.DataFrame(SI.fit_transform(train_data_x))
x_test = pd.DataFrame(SI.transform(test_data))

In [8]:
# name columns removed by the function SimpleImputer

x_train.columns = train_data_x.columns
x_test.columns = test_data.columns

# define the indices

x_train.index = train_data_x.index
x_test.index = test_data.index
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186523 entries, 184054 to 73018
Data columns (total 40 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   UniqueID                             186523 non-null  object
 1   disbursed_amount                     186523 non-null  object
 2   asset_cost                           186523 non-null  object
 3   ltv                                  186523 non-null  object
 4   branch_id                            186523 non-null  object
 5   supplier_id                          186523 non-null  object
 6   manufacturer_id                      186523 non-null  object
 7   Current_pincode_ID                   186523 non-null  object
 8   Date.of.Birth                        186523 non-null  object
 9   Employment.Type                      186523 non-null  object
 10  DisbursalDate                        186523 non-null  object
 11  State_ID              

In [9]:
# delete less significant columns 

drop_cols = ['DisbursalDate', 'PERFORM_CNS.SCORE.DESCRIPTION', 'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', \
             'branch_id', 'supplier_id', 'MobileNo_Avl_Flag', 'Aadhar_flag']

x_train = x_train.drop(drop_cols, axis = 1)
x_test = x_test.drop(drop_cols, axis = 1)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186523 entries, 184054 to 73018
Data columns (total 32 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   UniqueID                             186523 non-null  object
 1   disbursed_amount                     186523 non-null  object
 2   asset_cost                           186523 non-null  object
 3   ltv                                  186523 non-null  object
 4   manufacturer_id                      186523 non-null  object
 5   Current_pincode_ID                   186523 non-null  object
 6   Date.of.Birth                        186523 non-null  object
 7   Employment.Type                      186523 non-null  object
 8   State_ID                             186523 non-null  object
 9   Employee_code_ID                     186523 non-null  object
 10  PAN_flag                             186523 non-null  object
 11  VoterID_flag          

In [10]:
# encode features as an array

hot_encoder = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
OH_train = pd.DataFrame(hot_encoder.fit_transform(x_train[['Employment.Type']]))
OH_test = pd.DataFrame(hot_encoder.transform(x_test[['Employment.Type']]))

In [11]:
# define the indices

OH_train.index = x_train.index
OH_test.index = x_test.index

# remove the raw categorical variable Employment.Type

x_train = x_train.drop(['Employment.Type'], axis = 1)
x_test = x_test.drop(['Employment.Type'], axis = 1)

# replace the dropped categorical variable

x_train = pd.concat([x_train, OH_train], axis = 1)
x_test = pd.concat([x_test, OH_test], axis = 1)

In [12]:
# convert data type to integer

x_train = x_train.astype('int64')
x_test = x_test.astype('int64')

x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186523 entries, 184054 to 73018
Data columns (total 33 columns):
 #   Column                               Non-Null Count   Dtype
---  ------                               --------------   -----
 0   UniqueID                             186523 non-null  int64
 1   disbursed_amount                     186523 non-null  int64
 2   asset_cost                           186523 non-null  int64
 3   ltv                                  186523 non-null  int64
 4   manufacturer_id                      186523 non-null  int64
 5   Current_pincode_ID                   186523 non-null  int64
 6   Date.of.Birth                        186523 non-null  int64
 7   State_ID                             186523 non-null  int64
 8   Employee_code_ID                     186523 non-null  int64
 9   PAN_flag                             186523 non-null  int64
 10  VoterID_flag                         186523 non-null  int64
 11  Driving_flag                       

In [13]:
# build the model

model = LogisticRegression(random_state = 0, max_iter = 500, solver = 'liblinear')

# fitting of the model

model.fit(x_train, y_train)

# prediction

prediction = model.predict(x_test)

# calculate the accuracy of the model

accuracy = 1 - mae(y_test,prediction)
print("The accuracy is", accuracy)

The accuracy is 0.7799961399069288


In [14]:
# estimated default probabilities

model.predict_proba(x_test)

array([[0.89773215, 0.10226785],
       [0.72600715, 0.27399285],
       [0.77246678, 0.22753322],
       ...,
       [0.76347604, 0.23652396],
       [0.90943113, 0.09056887],
       [0.80464212, 0.19535788]])

In [15]:
# confusion matrix

cm = confusion_matrix(y_test, prediction)
print(cm)

[[36372     6]
 [10253     0]]
